<a href="https://colab.research.google.com/github/Meenalsh72/RFM_Analysis/blob/main/RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description:

I performed RFM analysis on Online retail store data using Python to segment customers based on their purchasing behavior. This helped identify customer loyalty and spending habits, enabling personalized marketing strategies.

**Purpose:**

- Understand customer behavior through data.
- Segment customers into groups based on their RFM scores.
- Help marketers design better marketing campaigns by focusing on specific customer segments.

**Technique:**

- Data Cleaning: Prepared the retail store data for analysis.
- RFM Scoring: Calculated RFM values for each customer using Python.
- Customer Segmentation: Grouped customers into segments based on their RFM scores.
- Insights and Recommendations: Derived actionable insights for marketing and decision-making.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import plotly.colors
from datetime import datetime


Here, I used a dataset from an online E-commerce company that contains information about customer purchase history.

In [30]:
data = pd.read_csv("/content/online_retail.csv")
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#Data Cleaning

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


removing null values from CustomerID  

In [32]:
data.dropna(subset=['CustomerID'],inplace=True)



*   convert inovicedate to datetime
*   adding a column of totalamount



In [33]:

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['TotalAmount'] = data['Quantity']*data['UnitPrice']

converting date years to 2023 and 2024

In [34]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Function to update year
def update_year(date):
    if date.year == 2010:
        return date.replace(year=2023)
    elif date.year == 2011:
        return date.replace(year=2024)
    return date

# Apply function to 'dates' column
data['InvoiceDate'] = data['InvoiceDate'].apply(update_year)

# Calculating RFM score

Setting the reference date to today to calculate how recently a customer made their last purchase based on their most recent transaction.

In [35]:
reference_date = pd.Timestamp(datetime.now().date())
print(reference_date)  # Example output: 2024-12-19 00:00:00

2024-12-20 00:00:00


Creating a dataframe names as 'rfm' which calculates Recency, frequncy and Monetary by grouping the data wrt to customer id from the data table

In [36]:
rfm = data.groupby('CustomerID').agg({
'InvoiceDate':lambda x : (reference_date - x.max()).days, #recency
'InvoiceNo': 'count', #freqeuncy
'TotalAmount':'sum'}) #monetary

Renaming column names in rfm dataframe as Recency, Frequency and Monetary

In [37]:
rfm.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency','TotalAmount':'Monetary'},inplace=True)

In the given code, RFM (Recency, Frequency, Monetary) scores are calculated to evaluate customer behavior. First, the data is divided into three features: Recency (days since last purchase), Frequency (number of purchases), and Monetary (total spending). The quantiles (25%, 50%, and 75%) for each feature are calculated to create thresholds for scoring. The `RScore` function assigns scores based on these quantiles, where customers with values closer to the lower quartile receive a lower score and those closer to the higher quartile receive a higher score. The scores for each feature (R, F, M) are then applied to the customer data, providing an overall assessment of customer behavior based on their recency, frequency, and spending habits.

In [38]:
# Define quantiles
quantiles = rfm.quantile(q=[0.25, 0.50, 0.75])

# Assign RFM scores
def RScore(x, p, d):
    if p == 'Recency':
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.50]:
            return 3
        elif x <= d[p][0.75]:
            return 2
        else:
            return 1
    else:  # For Frequency and Monetary
        if x <= d[p][0.25]:
            return 1
        elif x <= d[p][0.50]:
            return 2
        elif x <= d[p][0.75]:
            return 3
        else:
            return 4

# Apply RFM scoring
rfm['R'] = rfm['Recency'].apply(RScore, args=('Recency', quantiles,))
rfm['F'] = rfm['Frequency'].apply(RScore, args=('Frequency', quantiles,))
rfm['M'] = rfm['Monetary'].apply(RScore, args=('Monetary', quantiles,))

# Review the updated DataFrame
rfm

,Recency,Frequency,Monetary,R,F,M
CustomerID,,,,,,
12346.0,336,2,0.00,1,1,1
12347.0,12,182,4310.00,4,4,4
12348.0,85,31,1797.24,2,2,4
12349.0,28,73,1757.55,3,3,4
12350.0,321,17,334.40,1,1,2
...,...,...,...,...,...,...
18280.0,287,10,180.60,1,1,1
18281.0,190,7,80.82,1,1,1
18282.0,17,13,176.60,4,1,1


 Segmenting customers based on RScore as low, medium, and high-value customers.

In [53]:
# Ensure R, F, and M columns are integers
rfm['R'] = rfm['R'].fillna(0).astype(int)
rfm['F'] = rfm['F'].fillna(0).astype(int)
rfm['M'] = rfm['M'].fillna(0).astype(int)

# Create the 'segments' column by concatenating R, F, and M as strings
rfm['segments'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

# Create the 'Score' column by summing R, F, and M
rfm['Score'] = rfm[['R', 'F', 'M']].sum(axis=1)

# Review the updated DataFrame
rfm.head()

,Recency,Frequency,Monetary,R,F,M,segments,Score,rfm_segment_labels,RFM_Customer_segments
CustomerID,,,,,,,,,,
12346.0,336,2,0.00,1,1,1,111,3,Low-Value,Lost
12347.0,12,182,4310.00,4,4,4,444,12,High-Vatue,VIP/Loyal
12348.0,85,31,1797.24,2,2,4,224,8,Mid-Value,Potential Loyal
12349.0,28,73,1757.55,3,3,4,334,10,High-Vatue,VIP/Loyal
12350.0,321,17,334.40,1,1,2,112,4,Low-Value,Can't Lose


In [40]:
segment_labels = ['Low—Value', 'Mid—Vatue', 'High—Value']
def assign_segment(score):
    if score < 5:
        return 'Low-Value'
    elif score < 9:
        return 'Mid-Value'
    else:
        return 'High-Vatue'

rfm['rfm_segment_labels'] = rfm['Score'].apply(assign_segment)
rfm. head()


,Recency,Frequency,Monetary,R,F,M,segments,Score,rfm_segment_labels
CustomerID,,,,,,,,,
12346.0,336,2,0.00,1,1,1,111,3,Low-Value
12347.0,12,182,4310.00,4,4,4,444,12,High-Vatue
12348.0,85,31,1797.24,2,2,4,224,8,Mid-Value
12349.0,28,73,1757.55,3,3,4,334,10,High-Vatue
12350.0,321,17,334.40,1,1,2,112,4,Low-Value


Counting the number of customers in each segment.

In [55]:
# Count the occurrences of each RFM segment label
segments_counts = rfm['rfm_segment_labels'].value_counts().reset_index()

# Rename the columns for clarity
segments_counts.columns = ['RFM_segment', 'Count']

# Sort by the 'RFM_segment' column
segments_counts = segments_counts.sort_values('Count')
segments_counts

,RFM_segment,Count
2,Low-Value,781
1,High-Vatue,1692
0,Mid-Value,1899


Analyzing the number of customers falling in each segment with the help of a bar chart.

In [56]:
import plotly.express as px

# Create a bar chart using Plotly Express
fig = px.bar(segments_counts,
             x='RFM_segment',          # The RFM segment column for the x-axis
             y='Count',                # The count column for the y-axis
             title='Customer Distribution by RFM segment',  # Title of the chart
             labels={'RFM_segment': 'RFM Segment', 'Count': 'Number of Customers'},  # Axis labels
             color='RFM_segment',      # Corrected color argument
             color_discrete_sequence=px.colors.qualitative.Pastel)  # Color palette

# Show the plot
fig.show()

Segmenting customers further into five categories based on RFM score: VIP/Loyal, Potential Loyal, At Risk, Can't Lose, and Lost. Also Counting the number of customers in each segment to analyze engagement levels and tailor marketing strategies.

In [43]:
rfm['RFM_Customer_segments'] = ''
rfm.loc[rfm['Score'] >=9, 'RFM_Customer_segments'] = 'VIP/Loyal'
rfm.loc[(rfm['Score'] >=6) & (rfm['Score'] <9) , 'RFM_Customer_segments'] = 'Potential Loyal'
rfm.loc[(rfm['Score'] >=5) & (rfm['Score'] <6) , 'RFM_Customer_segments'] = 'At Risk Customers'
rfm.loc[(rfm['Score'] >=4) & (rfm['Score'] <5) , 'RFM_Customer_segments'] = 'Can\'t Lose'
rfm.loc[(rfm['Score'] >=3) & (rfm['Score'] <4) , 'RFM_Customer_segments'] = 'Lost'
segments_counts = rfm['RFM_Customer_segments'].value_counts().sort_index()



here counting the combinations of `rfm_segment_labels` and `RFM_Customer_segments`, showing how many customers fall into each combination. It then sorts the results to display the most common combinations first.

In [44]:
segment_product_counts = rfm.groupby(['rfm_segment_labels', 'RFM_Customer_segments']).size().reset_index(name = 'Count')
segment_product_counts = segment_product_counts.sort_values('Count', ascending = False)

creating a treemap to display the distribution of customers across different combinations of rfm_segment_labels and RFM_Customer_segments. It shows the count of customers in each combination, uses different colors for each rfm_segment_labels.

In [45]:
fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['rfm_segment_labels', 'RFM_Customer_segments'],
                                         values = 'Count',
                                         color = 'rfm_segment_labels',
                                         color_discrete_sequence = px.colors.qualitative.Pastel,
                                         title='RFM Customer Segemtns by values')
fig_treemap_segment_product.show()

visualizing the distribution for the VIP/Loyal segment
but why VIP/Loyal segment?
- **Targeted Analysis:** Focuses on VIP/Loyal customers to identify unique behaviors, as they are highly valuable.  
- **Retention Strategies:** Helps design strategies to retain and engage this group.  
- **Marketing Insights:** Provides insights to encourage frequent purchases if Monetary value is high but Recency is low.   


(Similar analysis can be conducted for different segments to understand their distribution and tailor customized strategies for them.)

In [46]:
vip_segment = rfm[rfm['RFM_Customer_segments']== 'VIP/Loyal']

In [62]:

fig = go.Figure()
fig.add_trace(go.Box(y=vip_segment['Recency'], name='Recency'))
fig.add_trace(go.Box(y=vip_segment['Frequency'], name='Frequency'))
fig.add_trace(go.Box(y=vip_segment['Monetary'], name='Monetary'))
fig.update_layout(
    yaxis=dict(range=[0, 10000])
)
fig.show()




The correlation matrix helps analyze relationships between Recency, Frequency, and Monetary values for VIP/Loyal customers. Understanding these correlations provides insights to refine strategies and enhance engagement with this valuable segment.

In [63]:
correlation_matrix = vip_segment[['R','F','M']].corr()

Creating a heatmap of the correlation matrix visualizes the relationships between Recency, Frequency, and Monetary values for the VIP/Loyal (Champion) segment. This helps identify strong or weak correlations, making it easier to interpret patterns and dependencies that can guide targeted decision-making for this valuable customer group.

In [65]:
fig_heatmap = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')
))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within VIP/Loyal segments')
fig_heatmap.show()

creating a bar chart comparing the number of customers in different RFM segments. The **VIP/Loyal (Champions)** segment is highlighted in a distinct color to emphasize its significance. The visualization helps analyze segment distributions and identify high-priority customer groups, aiding in strategic focus and resource allocation.

In [50]:
pastel_colors = plotly.colors.qualitative.Pastel
fig = go.Figure(data = [go.Bar(x=segments_counts.index, y=segments_counts.values,
                               marker = dict(color=pastel_colors))])
vip_color = 'rgb(158,202,225)'
fig.update_traces(marker_color=[vip_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segments_counts.index) ],
                   marker_line_color = 'rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title='Comparision of RFM Segments',
                  xaxis_title='RFM Segmentd',
                  yaxis_title='Number of Customers',
                  showlegend=False)
fig.show()

A grouped bar chart to compare the average Recency, Frequency, and Monetary scores for each RFM customer segment. Each bar represents the average score for one of the RFM dimensions within a specific segment, providing insights into how customer behavior differs across segments. This analysis helps identify the strengths and weaknesses of each segment, guiding targeted marketing and engagement strategies.

In [51]:
segment_scores = rfm.groupby('RFM_Customer_segments')[['R','F','M']].mean().reset_index()
fig = go.Figure()

#Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_segments'],
    y=segment_scores['R'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'

))
#Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_segments'],
    y=segment_scores['F'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'

))
#Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_segments'],
    y=segment_scores['M'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'

))

#update the layout
fig.update_layout(
    title='Comaprision of RFM Segment based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()

Conclusion: The RFM analysis provided actionable insights into customer behavior, enabling the identification of high-value VIP/Loyal customers and segments requiring re-engagement. These findings can guide targeted marketing strategies, enhance customer retention, and maximize business growth.